# Semantic reranking with a Hugging Face model

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/elastic/elasticsearch-labs/blob/main/notebooks/search/12-semantic-reranking-elastic-rerank.ipynb)

In this notebook you'll learn how to implement semantic reranking in Elasticsearch using the built-in [Elastic Rerank model](https://www.elastic.co/guide/en/machine-learning/master/ml-nlp-rerank.html). You'll also learn about the `retriever` abstraction, a simpler syntax for crafting queries and combining different search operations.

You will:

- Create an inference endpoint to manage your `rerank` task. This will download and deploy the Elastic Rerank model.
- Query your data using the `text_similarity_rerank` retriever, leveraging the Elastic Rerank model.

## 🧰 Requirements

For this example, you will need:

- An Elastic deployment:

  - We'll be using [Elastic Cloud](https://www.elastic.co/guide/en/cloud/current/ec-getting-started.html) for this example (available with a [free trial](https://cloud.elastic.co/registration?onboarding_token=vectorsearch&utm_source=github&utm_content=elasticsearch-labs-notebook))

- Elasticsearch 8.17.0 or above, or [Elasticsearch serverless](https://www.elastic.co/elasticsearch/serverless)

- A 4GB ML node

> ℹ️ Deploying the Elastic Rerank model in combination with ELSER (or other hosted models) requires at minimum 8GB ML node. The current max size for trial ML nodes is 4GB (defaults to 1GB).

## Install packages

This will take a couple of minutes.


In [ ]:
!pip install -qU elasticsearch

## Import packages

In [ ]:
from elasticsearch import Elasticsearch, helpers, exceptions
from urllib.request import urlopen
from getpass import getpass
import json
import time

## Initialize Elasticsearch Python client

You need to connect to a running Elasticsearch instance. In this example we're using an Elastic Cloud deployment.

In [5]:
# https://www.elastic.co/search-labs/tutorials/install-elasticsearch/elastic-cloud#finding-your-cloud-id
ELASTIC_CLOUD_ID = getpass("Elastic Cloud ID: ")

# https://www.elastic.co/search-labs/tutorials/install-elasticsearch/elastic-cloud#creating-an-api-key
ELASTIC_API_KEY = getpass("Elastic Api Key: ")

# Create the client instance
client = Elasticsearch(
    # For local development
    # hosts=["http://localhost:9200"]
    cloud_id=ELASTIC_CLOUD_ID,
    api_key=ELASTIC_API_KEY,
)

Elastic Cloud ID: ··········
Elastic Api Key: ··········


## Test connection

Confirm that the Python client has connected to your Elasticsearch instance with this test.



In [ ]:
print(client.info())

## Enable Telemetry

Knowing that you are using this notebook helps us decide where to invest our efforts to improve our products. We would like to ask you that you run the following code to let us gather anonymous usage statistics. See telemetry.py for details. Thank you!

In [ ]:
!curl -O -s https://raw.githubusercontent.com/elastic/elasticsearch-labs/main/telemetry/telemetry.py
from telemetry import enable_telemetry

es_client = enable_telemetry(client, "12-semantic-reranking-elastic-rerank")

## Upload sample data

This examples uses a small dataset of movies.

In [7]:
url = "https://raw.githubusercontent.com/elastic/elasticsearch-labs/main/notebooks/search/movies.json"
response = urlopen(url)

# Load the response data into a JSON object
data_json = json.loads(response.read())

# Prepare the documents to be indexed
documents = []
for doc in data_json:
    documents.append(
        {
            "_index": "movies",
            "_source": doc,
        }
    )

# Use helpers.bulk to index
helpers.bulk(client, documents)

print("Done indexing documents into `movies` index!")
time.sleep(3)

Done indexing documents into `movies` index!


## Create inference endpoint

Next we'll create an inference endpoint for the `rerank` task to deploy and manage our model and, if necessary, spin up the necessary ML resources behind the scenes.

In [ ]:
try:
    client.inference.delete(inference_id="my-elastic-reranker")
except exceptions.NotFoundError:
    # Inference endpoint does not exist
    pass

try:
    client.options(
        request_timeout=60, max_retries=3, retry_on_timeout=True
    ).inference.put(
        task_type="rerank",
        inference_id="my-elastic-reranker",
        inference_config={
            "service": "elasticsearch",
            "service_settings": {
                "model_id": ".rerank-v1",
                "num_threads": 1,
                "adaptive_allocations": {
                    "enabled": True,
                    "min_number_of_allocations": 1,
                    "max_number_of_allocations": 4,
                },
            },
        },
    )
    print("Inference endpoint created successfully")
except exceptions.BadRequestError as e:
    if e.error == "resource_already_exists_exception":
        print("Inference endpoint created successfully")
    else:
        raise e

ObjectApiResponse({'inference_id': 'my-msmarco-minilm-model', 'task_type': 'rerank', 'service': 'elasticsearch', 'service_settings': {'num_allocations': 1, 'num_threads': 1, 'model_id': 'cross-encoder__ms-marco-minilm-l-6-v2'}, 'task_settings': {'return_documents': True}})

Run the following command to confirm your inference endpoint is deployed.

In [ ]:
client.inference.get().body


⚠️ When you deploy your model, you might need to sync your ML saved objects in the Kibana (or Serverless) UI.
Go to **Trained Models** and select **Synchronize saved objects**.

## Lexical queries

First let's use a `standard` retriever to test out some lexical (or full-text) searchs and then we'll compare the improvements when we layer in semantic reranking.

### Lexical match with `query_string` query

Let's say we vaguely remember that there is a famous movie about a killer who eats his victims. For the sake of argument, pretend we've momentarily forgotten the word "cannibal".

Let's perform a [`query_string` query](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-query-string-query.html) to find the phrase "flesh-eating bad guy" in the `plot` fields of our Elasticsearch documents.

In [21]:
resp = client.search(
    index="movies",
    retriever={
        "standard": {
            "query": {
                "query_string": {
                    "query": "flesh-eating bad guy",
                    "default_field": "plot",
                }
            }
        }
    },
)

if resp["hits"]["hits"]:
    for hit in resp["hits"]["hits"]:
        title = hit["_source"]["title"]
        plot = hit["_source"]["plot"]
        print(f"Title: {title}\nPlot: {plot}\n")
else:
    print("No search results found")

No search results found


No results! Unfortunately we don't have any near exact matches for "flesh-eating bad guy". Because we don't have any more specific information about the exact phrasing in the Elasticsearch data, we'll need to cast our search net wider.

### Simple `match` query

This lexical query performs a standard keyword search for the term "crime" within the "plot" and "genre" fields of our Elasticsearch documents.

In [22]:
resp = client.search(
    index="movies",
    retriever={
        "standard": {
            "query": {"multi_match": {"query": "crime", "fields": ["plot", "genre"]}}
        }
    },
)

for hit in resp["hits"]["hits"]:
    title = hit["_source"]["title"]
    plot = hit["_source"]["plot"]
    print(f"Title: {title}\nPlot: {plot}\n")

Title: The Godfather
Plot: An organized crime dynasty's aging patriarch transfers control of his clandestine empire to his reluctant son.

Title: Goodfellas
Plot: The story of Henry Hill and his life in the mob, covering his relationship with his wife Karen Hill and his mob partners Jimmy Conway and Tommy DeVito in the Italian-American crime syndicate.

Title: The Silence of the Lambs
Plot: A young F.B.I. cadet must receive the help of an incarcerated and manipulative cannibal killer to help catch another serial killer, a madman who skins his victims.

Title: Pulp Fiction
Plot: The lives of two mob hitmen, a boxer, a gangster and his wife, and a pair of diner bandits intertwine in four tales of violence and redemption.

Title: Se7en
Plot: Two detectives, a rookie and a veteran, hunt a serial killer who uses the seven deadly sins as his motives.

Title: The Departed
Plot: An undercover cop and a mole in the police attempt to identify each other while infiltrating an Irish gang in South 

That's better! At least we've got some results now. We broadened our search criteria to increase the chances of finding relevant results.

But these results aren't very precise in the context of our original query "flesh-eating bad guy". We can see that "The Silence of the Lambs" is returned in the middle of the results set with this generic `match` query. Let's see if we can use our semantic reranking model to get closer to the searcher's original intent.

## Semantic reranker

In the following `retriever` syntax, we wrap our standard `match` query retriever in a `text_similarity_reranker`. This allows us to leverage the NLP model we deployed to Elasticsearch to rerank the results based on the phrase "flesh-eating bad guy".

In [ ]:
resp = client.search(
    index="movies",
    retriever={
        "text_similarity_reranker": {
            "retriever": {
                "standard": {
                    "query": {
                        "multi_match": {"query": "crime", "fields": ["plot", "genre"]}
                    }
                }
            },
            "field": "plot",
            "inference_id": "my-elastic-reranker",
            "inference_text": "flesh-eating bad guy",
        }
    },
)

for hit in resp["hits"]["hits"]:
    title = hit["_source"]["title"]
    plot = hit["_source"]["plot"]
    print(f"Title: {title}\nPlot: {plot}\n")

Title: The Silence of the Lambs
Plot: A young F.B.I. cadet must receive the help of an incarcerated and manipulative cannibal killer to help catch another serial killer, a madman who skins his victims.

Title: Pulp Fiction
Plot: The lives of two mob hitmen, a boxer, a gangster and his wife, and a pair of diner bandits intertwine in four tales of violence and redemption.

Title: Se7en
Plot: Two detectives, a rookie and a veteran, hunt a serial killer who uses the seven deadly sins as his motives.

Title: Goodfellas
Plot: The story of Henry Hill and his life in the mob, covering his relationship with his wife Karen Hill and his mob partners Jimmy Conway and Tommy DeVito in the Italian-American crime syndicate.

Title: The Dark Knight
Plot: When the menace known as the Joker wreaks havoc and chaos on the people of Gotham, Batman must accept one of the greatest psychological and physical tests of his ability to fight injustice.

Title: The Godfather
Plot: An organized crime dynasty's aging

Success! "The Silence of the Lambs" is our top result. Semantic reranking helped us find the most relevant result by parsing a natural language query, overcoming the limitations of lexical search that relies on keyword matching.

Semantic reranking enables semantic search in a few steps, without the need for generating and storing embeddings. This a great tool for testing and building hybrid search systems in Elasticsearch.

## Learn more

- [Elastic Rerank](https://www.elastic.co/guide/en/machine-learning/master/ml-nlp-rerank.html). Learn more about the Elastic Rerank model, including information about how the model is trained and how to deploy it in different environments.
- [Semantic Reranking overview](https://www.elastic.co/guide/en/elasticsearch/reference/current/semantic-reranking.html). A high level overview of semantic reranking in Elasticsearch.
- [`text_similarity_reranker` retriever ](https://www.elastic.co/guide/en/elasticsearch/reference/current/retriever.html#text-similarity-reranker-retriever). Detailed API syntax reference with examples.
- [`elasticsearch-labs` notebooks](https://github.com/elastic/elasticsearch-labs/tree/main/notebooks). Check our full catalogue of Python notebooks.